# **Exercice : manipulation de DataFrame**

À partir des datasets `population_communes.csv` et `surface_departements.csv`, créer un nouveau dataset qui contient une ligne par département, avec ces colonnes:

- la somme des "Population municipale" du département
- la somme des "Population totale" du département (pour l'explication de la distinction entre "Population municipale" et "Population totale", voir: https://www.insee.fr/fr/metadonnees/definition/c1270)
- la part (en pourcentage) de la population municipale par rapport à la population totale
- la part (en pourcentage) de la population (municipale) du département au sein de sa région
- la densité de la population (municipale) en nb d'habitants / km2

(Le dataset final devrait ressembler à `result-exo-cc.csv`)


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
#data check
df_pop_co = pd.read_csv('population_communes.csv')
df_pop_co.sort_values(by=['Code département'],ascending = True).head(5)

,Code département,Code canton,Code arrondissement,Code région,Nom de la commune,Code commune,Nom de la région,variable,value
53035,01,1.0,1,84,Saint-Denis-en-Bugey,345,Auvergne-Rhône-Alpes,Population municipale,2293
65760,01,16.0,4,84,Varambon,430,Auvergne-Rhône-Alpes,Population totale,543
27440,01,8.0,2,84,L' Abergement-Clémenciat,1,Auvergne-Rhône-Alpes,Population municipale,767
27441,01,8.0,2,84,L' Abergement-Clémenciat,1,Auvergne-Rhône-Alpes,Population totale,785
27446,01,1.0,1,84,L' Abergement-de-Varey,2,Auvergne-Rhône-Alpes,Population municipale,243


In [11]:
#data check
df_surface = pd.read_csv('surface_departements.csv')
df_surface.sort_values(by=["code_insee"],ascending=True).head(5)

,code_insee,nom,surf_km2
52,01,Ain,5784.0
45,02,Aisne,7411.0
36,03,Allier,7379.0
95,04,Alpes-de-Haute-Provence,6993.0
87,05,Hautes-Alpes,5697.0


In [45]:
#join DF to transform them
df = df_pop_co.set_index('Code département').join(df_surface.set_index('code_insee'))
df2 = df.groupby([df.index,"variable","surf_km2"])["value"].sum().reset_index()
df2.head(5)

,level_0,variable,surf_km2,value
0,01,Population municipale,5784.0,638425
1,01,Population totale,5784.0,655171
2,02,Population municipale,7411.0,536136
3,02,Population totale,7411.0,549587
4,03,Population municipale,7379.0,339384


In [46]:
df['value'] = df['value'].astype(int)
df['surf_km2'] = df['surf_km2'].astype(str) #empêcher toute addition
df2 = df.groupby([df.index,'variable','nom'])['value'].sum().reset_index()
#df2 = df.groupby([df.index,'variable'])['value'].sum()

In [48]:
df2.head(5)

,level_0,variable,nom,value
0,01,Population municipale,Ain,638425
1,01,Population totale,Ain,655171
2,02,Population municipale,Aisne,536136
3,02,Population totale,Aisne,549587
4,03,Population municipale,Allier,339384


In [54]:
df2.pivot_table(values = "value", index = "nom" , columns = "variable" , dropna = True)

variable,Population municipale,Population totale
nom,,
Ain,638425,655171
Aisne,536136,549587
Allier,339384,349336
Alpes-Maritimes,1083704,1098539
Alpes-de-Haute-Provence,162565,167331
...,...,...
Vendée,670597,689496
Vienne,436069,447026
Vosges,369641,382328


In [64]:
df2['variable'][1]

'Population totale'

In [65]:
df2["pop_mun/tot"]= df2["value"][0] / df2["value"][1] 

In [66]:
df2

,level_0,variable,nom,value,pop_mun/tot
0,01,Population municipale,Ain,638425,0.97444
1,01,Population totale,Ain,655171,0.97444
2,02,Population municipale,Aisne,536136,0.97444
3,02,Population totale,Aisne,549587,0.97444
4,03,Population municipale,Allier,339384,0.97444
...,...,...,...,...,...
195,972,Population totale,Martinique,382294,0.97444
196,973,Population municipale,Guyane,269352,0.97444
197,973,Population totale,Guyane,271829,0.97444
198,974,Population municipale,La Réunion,852924,0.97444


In [25]:
df3 = df.groupby([df.index,'surf_km2'])['value'].sum().reset_index()
df3.head()

,level_0,surf_km2,value
0,01,5784.0,1293596
1,02,7411.0,1085723
2,03,7379.0,688720
3,04,6993.0,329896
4,05,5697.0,287255


# **Correction**

1. Group by de la table commune population : .reset_index() -> dataframe ...reset_index()['noms des colonnes à garder'] </br>
2. On obtient une table avec les départements et les informations nous intéressant 
3. Puis opérations classiques sur les colonnes